# Parabolic Focus Determination Analysis

This notebook performs the analysis of focus sweeps taken with the Parabolic_Focus_Determination_Data_Acq.ipynb notebook. It can be used for spectral and non-spectral direct imaging sequences. 


## Setup

In [ ]:
from lsst.summit.extras import SpectralFocusAnalyzer, NonSpectralFocusAnalyzer
import matplotlib.pyplot as plt
import pandas as pd 

import lsst_efd_client
import os
from astropy.time import Time

In [ ]:
%matplotlib inline

In [ ]:
location = os.environ["LSST_DDS_PARTITION_PREFIX"]
print(f'{location=}')
if location == "summit":
    client = lsst_efd_client.EfdClient("summit_efd")
elif location == "tucson":
    client = lsst_efd_client.EfdClient("tucson_teststand_efd")
else:
    raise ValueError(
        "Location does not match any valid options {summit|tucson}"
        )


## Test information

In this section the original focus values will be declared and sequence numbers will be printed to use in the Analysis Section. 

Introduce here the day of observation (Careful with the date change at midnight UTC), and filter and grating used. 

In [ ]:
dayObs= 20230117
filter_used = 'SDSSr_65mm'
grating_used= 'holo4_003'

In [ ]:
day_obs = str(dayObs)
time_start = Time(f'{day_obs[0:4]}-{day_obs[4:6]}-{day_obs[6:9]}T00:00:01', scale='utc', format='isot')
time_end = Time(f'{day_obs[0:4]}-{day_obs[4:6]}-{day_obs[6:9]}T23:59:00', scale='utc', format='isot')

log_messages = await client.select_time_series('lsst.sal.Script.logevent_logMessage', 
                                      ['message','salIndex'], 
                                      time_start, 
                                      time_end)

salindex = log_messages[log_messages['message'].str.contains(f"START -- Parabolic_Focus -- {filter_used} and {grating_used}")]['salIndex'][0]

script_messages = log_messages[log_messages['salIndex']==(salindex)]

### To print all script messages from EFD

In [ ]:
pd.set_option('max_colwidth', 1500, 'display.max_rows', None)

# If you want to see all the script messages, uncomment the next line
# script_messages

### To print the exposures ids

In [ ]:
expIds= script_messages[script_messages['message'].str.contains(f"Image expId")]
print(f'The exposures taken for the parabolic focus sequence on {dayObs} with {filter_used} and {grating_used} are') 
display(expIds)

### To declare the original focus offset for use in later analysis

In [ ]:
originalFocus = script_messages[script_messages['message'].str.contains(f"Original focus offset")]

In [ ]:
original_filter_focus = float((originalFocus.message[0])[originalFocus.message[0].find("filter")+len("filter")+2:originalFocus.message[0].find(",",originalFocus.message[0].find("filter"))])
original_disperser_focus = float((originalFocus.message[0])[originalFocus.message[0].find("disperser")+len("disperser")+2:originalFocus.message[0].find(",",originalFocus.message[0].find("disperser"))])
original_userApplied_focus = float((originalFocus.message[0])[originalFocus.message[0].find("userApplied")+len("userApplied")+2:originalFocus.message[0].find(",",originalFocus.message[0].find("userApplied"))])

In [ ]:
print(f' Original User Applied Focus {original_userApplied_focus:0.3f} mm \n'
      f' Original Disperser Focus {original_disperser_focus:0.3f} mm \n'
      f' Original Filter Focus {original_filter_focus:0.3f} mm')

## Analysis

### Spectral Data
Skip to the `Imaging Data - Non Spectral` section below, if only dealing with images. 

#### Declare day of observation and image sequences.

dayObs format is yyyymmdd. <br>
SeqNums is a list of integers, containing the first and last image sequence of the focus sweep. 

In [ ]:
print(f'Observation day is {dayObs} \nIf you skipped the previous section and want to edit dayObs, please uncomment and edit the following cell')

In [ ]:
# dayObs = 20210323

Enter the image sequence numbers (+1 as the range function by default does not include the last value)

In [ ]:
seqNums = [s for s in range(286, 294+1)]
print(f'Observation day is {dayObs} \n'
      f'Sequence numbers are {seqNums}')

In case one or more of the images from the sequence are invalid, you can drop them from the seqNums array. Uncomment the cell below after you have replaced the images_to_discard list.    

In [ ]:
# images_to_discard = [286,288]
# for k in range(len(images_to_discard)):
#     try:
#         seqNums.remove(images_to_discard[k])
#     except:
#         print(f'{images_to_discard[k]} image not in original seqNums list')
        
print(f'\nObservation day is {dayObs} \n'
      f'New sequence numbers are {seqNums}')

#### Run the Spectral Focus Analyzer

The `SpectralFocusAnalyzer` takes slices through the spectrum at y-offsets, fitting a Gaussian to each spectrum slice to measure its width.
For each offset distance, fit a parabola to the fitted spectral widths and return the hexapod position at which the best focus was achieved for each offset. 
The number of slices and their distances to the zero-order star can be customized by calling `setSpectrumBoxOffsets().`

In [ ]:
focusAnalyzer = SpectralFocusAnalyzer()

# The numerical values in the square brackets inside the `setSpectrumBoxOffsets()` need to be manually adjusted so a Gaussian fit falls in the center of each spectrum slice. 
# Each number represents the distance, in pixels, from the zero orden spectrum. 

focusAnalyzer.setSpectrumBoxOffsets([500, 750, 1000, 1250])

focusAnalyzer.getFocusData(dayObs, seqNums, doDisplay=True)
focusAnalyzer.fitDataAndPlot()

#### Results

The value obtained in the previous analysis must be corrected corrected with the user_applied focus offset (from the analysis above) as well as the value in the currently loaded configuration file (in ts_config_latiss) for the ATSpectrograph CSC.

In [ ]:
result_focus_analyzer = 

In [ ]:
offset = result_focus_analyzer - original_userApplied_focus
new_value_disperser_focus = original_disperser_focus + offset

In [ ]:
print(f'The new value in the config file for the disperser focus is {new_value_disperser_focus:0.3f} mm')

### Imaging Data - Non spectral
Do not execute this section if you are dealing with spectra: please proceed to section above `Spectral Data`

#### Declare day of observation and image sequences.

dayObs format is yyyymmdd. <br>
SeqNums is a list of integers, containing the first and last image of the focus sweep sequence. 

In [ ]:
print(f'Observation day is {dayObs} \nIf you skipped the previous section and want to edit dayObs, please uncomment and edit the following cell')

In [ ]:
# dayObs = 20210323

Enter the image sequence numbers

In [ ]:
seqNums = [s for s in range(286, 294+1)]
print(f'Observation day is {dayObs} \n'
      f'Sequence numbers are {seqNums}')

In case one or more of the images from the sequence are invalid, you can drop them from the seqNums array. Uncomment the cell below after you have replaced the images_to_discard values.   

In [ ]:
# images_to_discard = [286,288]
# for k in range(len(images_to_discard)):
#     try:
#         seqNums.remove(images_to_discard[k])
#     except:
#         print(f'{images_to_discard[k]} image not in original seqNums list')
        
print(f'\nObservation day is {dayObs} \n'
      f'New sequence numbers are {seqNums}')

#### Run the Focus Analyzer

For each image, the `NonSpectralFocusAnalyzer()` measures the FWHM of the main star and the 50/80/90% encircled energy radii, and fit a parabola to get the position of best focus.

In [ ]:
focusAnalyzer = NonSpectralFocusAnalyzer()

focusAnalyzer.getFocusData(dayObs, seqNums, doDisplay=True)
focusAnalyzer.fitDataAndPlot()

#### Results

The result obtained in the previous analysis must be corrected with the user_applied focus offset (from the analysis above) as well as the value in the currently loaded configuration file (in ts_config_latiss) for the ATSpectrograph CSC. 

In [ ]:
result_focus_analyzer = 

In [ ]:
offset = result_focus_analyzer - original_userApplied_focus
new_value_filter_focus = original_filter_focus + offset

In [ ]:
print(f'The new value in the config file for the filter focus should be {new_value_filter_focus:0.3f} mm')